### Piecemeal Clustering
A clustering approach without prior knowledge about data even number of clusters. It uses similarity and density of data to define number of clusters. 

### Reading the Paper
#### Introduction
- No noise: KMeans, SOM
- Density-based: DBSCAN
- Precedents: trial and errors, combinations of multiple methods
- What is lithofaces, 
- The algorithm utilizes the density-based clustering combining the concepts of hierachical clustering, model-based unsupervised learning and density-based data clustering. 

### Reference:
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9980364

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from scipy.cluster import hierarchy as sch

In [2]:
X, y = datasets.load_iris(as_frame=True, return_X_y=True)

In [6]:
X = X.to_numpy()
y = y.to_numpy()

In [7]:
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Shape of X: (150, 4)
Shape of y: (150,)


In [8]:
def merge_cluster(clusters, i, j):
    for t in range(len(clusters)):
        if clusters[t] == j:
            clusters[t] = clusters[i]
    return clusters

In [9]:
def ind_to_pair(d, index):
    b = 1 - (2 * d) 
    i = (-b - np.sqrt(b ** 2 - 8 * index)) // 2
    j = index + i * (b + i + 2) // 2 + 1
    return (int(i), int(j))  

In [10]:
def compute_dist_threshold(dmatrix, cutoff: float):
    dmin = min(dmatrix)
    dmax = max(dmatrix)
    return (1 - (dmin / dmax)) * cutoff

def compute_sim_threshold(smatrix, cutoff: float, diff_factor: float):
    dmin = min(smatrix)
    dmax = max(smatrix)
    return (1 - (dmin / dmax)) * cutoff * diff_factor

In [11]:
def compute_cluster_distances(dmatrix, smatrix, diff_factor: float):
    dmax = max(dmatrix)
    smax = max(smatrix)

    dmatrix = dmatrix / dmax
    dmatrix = dmatrix ** 2
    smatrix = smatrix / smax
    smatrix = diff_factor * (smatrix**2)

    return np.sqrt(dmatrix + smatrix)

In [15]:
def pre_clustering(X, cutoff: float, diff_factor: float):
    dmatrix = sch.distance.pdist(X, metric="euclidean")
    smatrix = sch.distance.pdist(X, metric="cosine")
    clusters = np.arange(0, len(X), 1)
    dt = compute_dist_threshold(dmatrix, cutoff)
    st = compute_sim_threshold(smatrix, cutoff, diff_factor)
    tds = np.sqrt(dt**2 + st**2)
    cd = compute_cluster_distances(dmatrix, smatrix, diff_factor)
    dim = len(X)

    print(f"Minimum distance: {tds}")
    sorted_cd = np.argsort(cd)
    
    for index in sorted_cd:
        d = cd[index]
        i, j = ind_to_pair(dim, index)
        if d <= tds:
            merge_cluster(clusters, i, j)

    return clusters


clusters = pre_clustering(X, 0.015, 10)

Minimum distance: 0.15074813431681336


In [47]:
def compute_cluster_center_mapping(names, X, y):
    data = np.column_stack((names, X, y))
    df = pd.DataFrame(data, columns=["cluster", "x1", "x2", "x3", "x4", "y"])
    df["cluster"] = df["cluster"].astype(int)

    df1 = df.groupby(by=["cluster"])[["x1", "x2", "x3", "x4"]].mean().sort_index()
    df2 = df.groupby(by=["cluster"])["y"].agg(pd.Series.mode).sort_index()
    return df1.to_numpy(), df2.to_numpy()

In [137]:
cluster_centers, cluster_mapping = compute_cluster_center_mapping(clusters, X, y)

In [138]:
print(f"Shape of Cluster Centers: {cluster_centers.shape}")

Shape of Cluster Centers: (5, 4)


In [139]:
def compute_decay(step, N):
    return 1 - (step / N)

def compute_radii(W):
    dmatrix = sch.distance.pdist(W, metric="euclidean")
    smatrix = sch.distance.pdist(W, metric="cosine")
    radius_d = (max(dmatrix) - min(dmatrix)) / 2
    radius_s = (max(smatrix) - min(smatrix)) / 2
    return radius_d, radius_s

def unit_distances(x, W):
    x = x.reshape(1, -1)
    W = W.copy()
    d = np.sqrt(np.sum((x - W)**2, axis=1))
    x = x / np.linalg.norm(x, axis=1)
    W = W.T / np.linalg.norm(W, axis=1)
    ### Same method as pdist
    s = 1 - x.dot(W)
    return d, s.reshape(-1)

def train_som(X, W, N, alpha_0):
    """This is the modifed version of training Self-organizing map
    Args:
    - X: input data shape (m, n)
    - W: weight matrix, in piecemeal, the weight matrix is the cluster centers
    - alpha_0: learning rate
    - N: number of iterations
    """
    W = W.copy()
    n_neurals = len(W)
    n_samples = len(X)
    print(f"Number of neurals: {n_neurals}")
    rd_0, rs_0 = compute_radii(W)
    for step in range(N):
        rd = rd_0 * compute_decay(step, N)
        rs = rs_0 * compute_decay(step, N)
        alpha = alpha_0 * compute_decay(step, N)

        x = X[np.random.choice(n_samples)]
        ud, us = unit_distances(x, W)
        for i, (cd, cs) in enumerate(zip(ud, us)):
            if cd < rd and cs < rs:
                i_d = np.exp((-cd**2) / (2*rd**2))
                i_s = np.exp((-cs**2) / (2*rs**2))
                influence = np.sqrt(i_d * i_s)
                W[i] += alpha * influence * (x - W[i])
    return W

In [140]:
cluster_centers

array([[5.006     , 3.428     , 1.462     , 0.246     ],
       [6.36101695, 2.92542373, 5.05254237, 1.79830508],
       [5.53214286, 2.63571429, 3.96071429, 1.22857143],
       [6.3       , 3.3       , 6.        , 2.5       ],
       [7.475     , 3.125     , 6.3       , 2.05      ]])

In [142]:
train_som(X, cluster_centers, 10000, 0.001)

Number of neurals: 5


array([[4.99974208, 3.42323295, 1.46683226, 0.24634987],
       [6.21259188, 2.88096088, 4.85312563, 1.66848272],
       [5.9417436 , 2.78130169, 4.46797915, 1.46189966],
       [6.39307002, 3.00715134, 5.30348633, 1.9480127 ],
       [6.80656279, 3.02795585, 5.61259458, 1.9681041 ]])